In [77]:
from glob import glob
import os
from collections import defaultdict
from pathlib import Path
from enum import Enum
import json
import pandas as pd
import seaborn as sns

In [78]:
UNATE_SYNTHESIS_PATH = '../experiments/synthesis/unates/'
DEPS_SYNTHESIS_PATH = '../experiments/synthesis/dependency/'
UNATE_AND_DEPS_SYNTHESIS_PATH = '../experiments/synthesis/unates_and_dependency/'
PLAIN_SYNTHESIS_PATH = '../experiments/synthesis/plain/'

In [79]:
class Status(Enum):
    ERROR = "Error"
    SUCCESS = "Success"
    NOT_FOUND = "Not Found"
    TIMEOUT = "Timeout"
    
    def __repr__(self):
        return str(self.value)
    
    def __str__(self):
        return str(self.value)

In [80]:
def get_all_benchmarks(path):
    hoa_files = glob(os.path.join(path, "*.hoa"))
    names = [
        Path(f).stem
        for f in hoa_files
    ]
    return names

print("Total benchmark with Unates Synthesis" , len(get_all_benchmarks(UNATE_SYNTHESIS_PATH)))
print("Total benchmark with Deps Synthesis" , len(get_all_benchmarks(DEPS_SYNTHESIS_PATH)))
print("Total benchmark with Unates & Deps Synthesis" , len(get_all_benchmarks(UNATE_AND_DEPS_SYNTHESIS_PATH)))
print("Total benchmark with Plain Synthesis" , len(get_all_benchmarks(PLAIN_SYNTHESIS_PATH)))

Total benchmark with Unates Synthesis 207
Total benchmark with Deps Synthesis 207
Total benchmark with Unates & Deps Synthesis 207
Total benchmark with Plain Synthesis 207


In [81]:
def extract_vars_from_str(x):
    if len(x) == 0:
        return []
    return x.split(",")

In [82]:
def benchmark_loader(path: str, name: str):
    base = {
        'Name': name,
    }
    
    json_path = os.path.join(path, name + ".json")
    
    if not os.path.exists(json_path):
        return {
            **base,
            'Status': Status.NOT_FOUND
        }
    
    with open(json_path, 'r') as f:
        content = f.readlines()
        try:
            benchmark_json = json.loads(content[0])
        except Exception as e:
            return {
                **base,
                'Status': Status.ERROR,
                'Error': content
            }
    
    # Generic Benchmark Name
    output_vars = benchmark_json['output_vars']
    is_completed = benchmark_json['is_completed']
    total_duration = benchmark_json['total_time']
    
    if not is_completed:
        return {
            **base,
            'Status': Status.TIMEOUT,
            'Is Completed': is_completed,
            'Total Duration': total_duration
        }
    
    # Automaton Build
    is_automaton_build = benchmark_json['automaton']['is_built']
    automaton_build_duration = benchmark_json['automaton']['build_duration']
    total_states = benchmark_json['automaton']['prune_total_states']
    total_edges = benchmark_json['automaton']['total_edges']
    total_output_vars = len(benchmark_json['output_vars'])
    
    # Synthesis
    realizability = benchmark_json['synthesis']['independent_strategy']['realizability']
    indeps_synthesis_duration = benchmark_json['synthesis']['independent_strategy']['duration']
    deps_synthesis_duration = benchmark_json['synthesis']['dependent_strategy']['duration']
    
    # Unateness
    skipped_unate = benchmark_json['unate']['skipped_unate']
    if skipped_unate:
        total_unate_duration = None
        automaton_postprocessing_duration = None
        removed_states = None
        removed_edges = None
    else:
        total_unate_duration = benchmark_json['unate']['total_unate_duration']
        automaton_postprocessing_duration = benchmark_json['unate']['automaton_postprocess_duration']
        removed_states = total_states - benchmark_json['unate']['total_states_after_unate']
        removed_edges = total_edges - benchmark_json['unate']['total_edges_after_unate']

    vars_positive_unates = { var: 0 for var in output_vars }
    vars_negative_unates = { var: 0 for var in output_vars }
    vars_unknown_unates = { var: 0 for var in output_vars }
    state_positive_unates = { state: 0 for state in range(total_states) }
    state_negative_unates = { state: 0 for state in range(total_states) }
    states_not_unates = { state: 0 for state in range(total_states) }
    states_unknown_unates = { state: 0 for state in range(total_states) }
    states_complement_duration = { state: 0 for state in range(total_states) }
    states_complement_failed = { state: None for state in range(total_states) }
    states_removed_edges = { state: 0 for state in range(total_states) }
    states_impacted_edges = { state: 0 for state in range(total_states) }
    
    for state_unateness in benchmark_json['unate'].get('unate_states',[]):
        state = state_unateness['state']
        state_positive_vars = extract_vars_from_str(state_unateness['positive_unate_variables'])
        state_negative_vars = extract_vars_from_str(state_unateness['negative_unate_variables'])
        state_not_unates = extract_vars_from_str(state_unateness['not_unate_variables'])
        state_unknown_unates = extract_vars_from_str(state_unateness['unknown_unate_variables'])
        assert set(state_positive_vars).isdisjoint(set(state_negative_vars))
        
        states_removed_edges[state] = state_unateness['removed_edges']
        states_impacted_edges[state] = state_unateness['impacted_edges']
        
        states_complement_duration[state] = state_unateness['complement_duration']
        states_complement_failed[state] = not state_unateness['complement_succeeded']
        
        for var in state_positive_vars:
            vars_positive_unates[var] += 1
            state_positive_unates[state] += 1
        
        for var in state_negative_vars:
            vars_negative_unates[var] += 1
            state_negative_unates[state] += 1
            
        for var in state_unknown_unates:
            vars_unknown_unates[var] += 1
            states_unknown_unates[state] += 1
    
    # Dependency
    skipped_dependency = benchmark_json['dependencies']['skipped_dependencies']
    find_dependency_duration = benchmark_json['dependencies']['total_duration']
    total_dependent_vars = 0
    total_independent_vars = 0
    for tested_dependency in benchmark_json['tested_dependencies']:
        is_dependent = tested_dependency['is_dependent']
        if is_dependent:
            total_dependent_vars += 1
        else:
            total_independent_vars += 1
    
    return {
        # Group 1: General
        **base,
        'Status': Status.SUCCESS,
        'Is Completed': is_completed,
        'Total Duration': total_duration,
        'Realizability': realizability,
        'Applied Unate': not skipped_unate,
        'Applied Dependency': not skipped_dependency,
        'Independent Strategy Gates': benchmark_json['synthesis']['independent_strategy']['total_gates'],
        'Independent Synthesis Duration': indeps_synthesis_duration,
        'Dependent Synthesis Duration': deps_synthesis_duration,
        
        # Group 2: Automaton
        'Is Automaton Built': is_automaton_build,
        'Automaton Build Duration': automaton_build_duration,
        'Total Output Vars': total_output_vars,
        'Original Total States': total_states,
        'Original Total Edges': total_edges,
        
        # Unate Impactness
        'Total Unate Duration': total_unate_duration,
        'Automaton Postprocessing Duration': automaton_postprocessing_duration,
        'Total Impacted Edges': sum(states_impacted_edges.values()),
        'Total Removed Edges': sum(states_removed_edges.values()),

        # Group 3: Unates by States
        'States Failed by Complement': sum(1 for val in states_complement_failed.values() if val),
        'Total Complement Duration': sum(val for val in states_complement_duration.values()),
        'Total State with Positive Unates': sum(1 for val in state_positive_unates.values() if val > 0),
        'Total State with Negative Unates': sum(1 for val in state_negative_unates.values() if val > 0),
        'Total State with Unknown Unates': sum(1 for val in states_unknown_unates.values() if val > 0),
        
        # Group 4: Unates by Vars
        'Total Positive Unates Vars (At least 1)': sum(1 for val in vars_positive_unates.values() if val > 0),
        'Total Negative Unates Vars (At least 1)': sum(1 for val in vars_negative_unates.values() if val > 0),
        'Total Positive Unates Vars (All)': sum(1 for val in vars_positive_unates.values() if val == total_states),
        'Total Negative Unates Vars (All)': sum(1 for val in vars_negative_unates.values() if val == total_states),
        
        # Dependency
        'Total Dependent Variables': total_dependent_vars,
        'Total Independent Variables': total_independent_vars,
        'Dependency Ratio': total_dependent_vars / total_output_vars,
        'Find Dependency Duration': find_dependency_duration,
        'Find Dependency Completed': find_dependency_duration != -1,
    }

In [83]:
unates_synthesis_df = pd.DataFrame([
    benchmark_loader(UNATE_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(UNATE_SYNTHESIS_PATH)
])
unates_synthesis_df.head(5)

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total State with Unknown Unates,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed
0,TwoCountersInRangeA4,Success,True,215.0,UNREALIZABLE,True,False,-1.0,69.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
1,ltl2dba05,Success,True,3.0,UNREALIZABLE,True,False,-1.0,1.0,-1.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
2,ltl2dba11,Success,True,2.0,UNREALIZABLE,True,False,-1.0,1.0,-1.0,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
3,TwoCountersDisButA6,Success,True,20421.0,UNREALIZABLE,True,False,-1.0,11998.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
4,TorcsSteeringSimple,Success,True,4.0,REALIZABLE,True,False,13.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False


In [84]:
dependents_synthesis_df = pd.DataFrame([
    benchmark_loader(DEPS_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(DEPS_SYNTHESIS_PATH)
])
dependents_synthesis_df.head(5)

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total State with Unknown Unates,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed
0,TwoCountersInRangeA4,Success,True,147.0,UNREALIZABLE,False,True,-1.0,40.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.375000,2.0,True
1,ltl2dba05,Success,True,5.0,REALIZABLE,False,True,135.0,3.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,True
2,ltl2dba11,Success,True,2.0,REALIZABLE,False,True,8.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,True
3,TwoCountersDisButA6,Success,True,9222.0,UNREALIZABLE,False,True,-1.0,3815.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,4.0,9.0,0.307692,47.0,True
4,TorcsSteeringSimple,Success,True,82.0,REALIZABLE,False,True,11.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.333333,0.0,True


In [85]:
plain_synthesis_df = pd.DataFrame([
    benchmark_loader(PLAIN_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(PLAIN_SYNTHESIS_PATH)
])
plain_synthesis_df.head(5)

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total State with Unknown Unates,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed
0,TwoCountersInRangeA4,Success,True,247.0,UNREALIZABLE,False,False,-1.0,97.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
1,ltl2dba05,Success,True,5.0,REALIZABLE,False,False,135.0,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
2,ltl2dba11,Success,True,3.0,REALIZABLE,False,False,8.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
3,TwoCountersDisButA6,Success,True,14905.0,UNREALIZABLE,False,False,-1.0,8810.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False
4,TorcsSteeringSimple,Success,True,4.0,REALIZABLE,False,False,13.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False


In [86]:
unates_and_deps_synthesis_df = pd.DataFrame([
    benchmark_loader(UNATE_AND_DEPS_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(UNATE_AND_DEPS_SYNTHESIS_PATH)
])
unates_and_deps_synthesis_df.head(5)

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total State with Unknown Unates,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed
0,TwoCountersInRangeA4,Success,True,123.0,UNREALIZABLE,True,True,-1.0,34.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.375000,2.0,True
1,ltl2dba05,Success,True,4.0,UNREALIZABLE,True,True,-1.0,1.0,-1.0,...,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.0,True
2,ltl2dba11,Success,True,2.0,UNREALIZABLE,True,True,-1.0,0.0,-1.0,...,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.0,True
3,TwoCountersDisButA6,Success,True,8178.0,UNREALIZABLE,True,True,-1.0,3512.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,4.0,9.0,0.307692,43.0,True
4,TorcsSteeringSimple,Success,True,129.0,REALIZABLE,True,True,11.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.333333,0.0,True


# Sanity Checks

## No Realiability Contradication

In [94]:
realizability_df = pd.merge(
    pd.merge(
        plain_synthesis_df[["Name", "Original Total States", "Total Output Vars", "Realizability"]],
        unates_and_deps_synthesis_df[["Name", "Realizability"]],
        suffixes=('_plain', '_all'),
        on='Name'
    ),
    pd.merge(
        unates_synthesis_df[["Name", "Realizability", "Total Impacted Edges"]],
        dependents_synthesis_df[["Name", "Realizability", "Total Dependent Variables"]],
        suffixes=('_unates', '_deps'),
        on='Name'
    ),
    on='Name'
)
realizability_df

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
0,TwoCountersInRangeA4,6.0,8.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
1,ltl2dba05,13.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,21.0,REALIZABLE,0.0
2,ltl2dba11,5.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,2.0,REALIZABLE,0.0
3,TwoCountersDisButA6,10.0,13.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,4.0
4,TorcsSteeringSimple,4.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,0.0,REALIZABLE,1.0
...,...,...,...,...,...,...,...,...,...
202,OneCounterGuiA1,7.0,9.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
203,EscalatorSmart,14.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,0.0,REALIZABLE,1.0
204,OneCounterGuiA0,6.0,9.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
205,ltl2dba20,19.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,12.0,REALIZABLE,0.0


In [95]:
different_realizability_df = realizability_df[
    (
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_deps']) |
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_plain']) | 
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_all']) | 
        (realizability_df['Realizability_all'] != realizability_df['Realizability_plain']) | 
        (realizability_df['Realizability_all'] != realizability_df['Realizability_deps']) | 
        (realizability_df['Realizability_deps'] != realizability_df['Realizability_plain']) 
    )
    & (realizability_df['Realizability_unates'].notna())
    & (realizability_df['Realizability_deps'].notna())
]
different_realizability_df

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
1,ltl2dba05,13.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,21.0,REALIZABLE,0.0
2,ltl2dba11,5.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,2.0,REALIZABLE,0.0
7,ltl2dba10,10.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,7.0,REALIZABLE,0.0
8,ltl2dba04,12.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,11.0,REALIZABLE,0.0
10,ltl2dba12,9.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,11.0,REALIZABLE,0.0
11,ltl2dba06,15.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,19.0,REALIZABLE,0.0
15,amba_decomposed_tsingle,21.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,5.0,REALIZABLE,0.0
23,ltl2dba07,33.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,55.0,REALIZABLE,0.0
24,ltl2dba13,11.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,12.0,REALIZABLE,0.0
29,ltl2dba17,28.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,4.0,REALIZABLE,0.0


In [96]:
different_realizability_df[different_realizability_df["Total Dependent Variables"] > 0]

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
37,SensorInit,8.0,17.0,REALIZABLE,NaN,REALIZABLE,0.0,REALIZABLE,9.0


In [97]:
different_realizability_df[different_realizability_df["Total Impacted Edges"] == 0]

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
37,SensorInit,8.0,17.0,REALIZABLE,NaN,REALIZABLE,0.0,REALIZABLE,9.0


## All DataFrame has same order

In [90]:
def verify_name_order(*dataframes):
    for i in range(1, len(dataframes)):
        if not dataframes[i - 1]['Name'].equals(dataframes[i]['Name']):
            return False
    return True

In [91]:
verify_name_order(
    unates_and_deps_synthesis_df,
    plain_synthesis_df,
    dependents_synthesis_df,
    unates_synthesis_df
)

True